^C
ERROR: Operation cancelled by user


In [1]:
from llama_index.core import VectorStoreIndex,ServiceContext,Document, SimpleDirectoryReader
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core import Settings
from llama_index.core import PromptTemplate
from pymongo import MongoClient
import pandas as pd
import numpy as np
import csv
import os

In [3]:
import os

os.environ["HUGGINGFACEHUB_API_TOKEN"] = "YOUR API KEY"

In [6]:
from llama_index.embeddings.langchain import LangchainEmbedding
from langchain.embeddings.huggingface import HuggingFaceBgeEmbeddings

embed_model = LangchainEmbedding(HuggingFaceBgeEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2"))

/mnt/disk1/sumdev/anaconda3/envs/exp1/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [13]:
system_prompt="""

Below are sql tables schemas paired with instruction that describes a task. 
Using valid SQLite, execute the query provided in the instructions. If a query result returns a blank result, return the reason why it does it. 
        Example:
        
        Input:
        ### Query: SELECT * FROM users as u Where u.email='sg@gmail.com';
        ### Database: users

        CREATE TABLE IF NOT EXISTS users (
                roll_id INTEGER PRIMARY KEY,
                name TEXT NOT NULL,
                email TEXT NOT NULL
            );
        INSERT INTO users (name, email) VALUES (?, ?)", ('soham', 'sg@gmail.com');
        INSERT INTO users (name, email) VALUES (?, ?)", ('soam', 'sdg@gmail.com');
        INSERT INTO users (name, email) VALUES (?, ?)", ('sm', 'asd@gmail.com');

        
        Context:
        The row ('soham', 'sg@gmail.com') has the email set as 'sg@gmail.com'
        
        Output:
        The row ('soham', 'sg@gmail.com') is returned

        Instruction:
        Execute user's query according to db schema provided, if there will be error output it
"""


query_prompt = PromptTemplate("<|USER|>{query_str}<|ASSISTANT|>")

In [20]:
system_prompt1 = """

You are a python coder with 10 years of experience and a master in coding. You have good cabilities in integration, webhooks, and automation

        
        Context:
        You will be given an error, draft an email and send to user defined gmail via python webhook, listen for the output via webhook and append it into database back again
        
        Output:
        '''
Here is the python script for you reference in case you need to implement a trigger
const { Trigger, Actions, Conditions } = require('trigger.dev');
const sqlQuery = 'SELECT * FROM example_table WHERE column = "some_value"';
const gmailCredentials = {...};
const webhookURL = 'https://example.com/webhook';

const trigger = new Trigger({
  name: 'SQL Query & Email Automation',
  event: {
    type: 'cron',
    cron: '*/15 * * * *', // Run every 15 minutes
  },
  on: async (event, context) => {
    const queryResult = await context.sql.query(sqlQuery);

    if (queryResult.length === 0) {
      const emailAction = new Actions.Gmail({
        apiKey: gmailCredentials,
        to: 'recipient@example.com',
        subject: 'Example subject',
        body: 'Example body',
      });

      const emailResponse = await emailAction.sendEmail();

      context.waitForEvent('gmail.email_replied', {
        threadId: emailResponse.threadId,
      });

      const reply = context.event.data;
      const parsedReply = parseReply(reply); // Custom function to parse the reply data

      const webhookAction = new Actions.Webhook({
        url: webhookURL,
        method: 'POST',
        body: parsedReply,
      });

      await webhookAction.run();
    }
  },
});

trigger.run();
'''

        Instruction:
        Execute user's query step by step, take break in between and produce accurate code"""

In [5]:
import torch

llm= HuggingFaceLLM(
context_window=4096,
max_new_tokens=2048,
generate_kwargs={"temperature":0.5, "do_sample": False},
system_prompt=system_prompt,
query_wrapper_prompt=query_prompt,
tokenizer_name="mistralai/Mistral-7B-Instruct-v0.2",
model_name="mistralai/Mistral-7B-Instruct-v0.2",
device_map="cuda:0")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
import pandas as pd

In [ ]:
csv_file ="/mnt/disk1/sumdev/sanyam/users.csv"
df = pd.read_csv(csv_file)
df = df.iloc[:,1:]
df

In [7]:
reader = SimpleDirectoryReader(input_files=["/mnt/disk1/sumdev/sanyam/SQL-Cheat-Sheet.txt"])
data = reader.load_data()

In [8]:
sevice_context=ServiceContext.from_defaults(chunk_size=1024, llm=llm,embed_model=embed_model)


/tmp/ipykernel_434012/2912935011.py:1: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  sevice_context=ServiceContext.from_defaults(chunk_size=1024, llm=llm,embed_model=embed_model)


In [9]:


index = VectorStoreIndex.from_documents(data,service_context=sevice_context)


In [10]:
query_engine = index.as_query_engine()

In [14]:
user_prompt = "SELECT * FROM users as u Where u.email='@gmail.com';"

In [15]:
answer = query_engine.query(system_prompt + user_prompt)

/mnt/disk1/sumdev/anaconda3/envs/exp1/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [16]:
print(answer)

The query 'SELECT * FROM users as u Where u.email='@gmail.com';' will return an empty result because the email address in the query does not match any record in the 'users' table. However, it's worth noting that SQLite does not support the '@' symbol in email addresses directly in the query. To search for emails containing the '@' symbol, you would need to use a wildcard character such as '%' to search for partial matches. For example, 'SELECT * FROM users as u Where u.email LIKE '%@gmail.com%';' would return all records where the email address contains '@gmail.com'. Alternatively, you could update the 'users' table to store email addresses in a normalized format, such as 'username@domain.com', to make queries easier.


In [18]:
new_prompt = f" Understand {answer} given by you, draft an email for email user <sanyam0605@gmail.com> resulting an email with defining whats the output, now take this message and write a code of python script containg this message , waiting for output by email id mentioned and appending that result into SQL database "

In [21]:
answer1 = query_engine.query(system_prompt1 + new_prompt)

/mnt/disk1/sumdev/anaconda3/envs/exp1/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [22]:
print(answer1)

To draft an email for the user <sanyam0605@gmail.com>, you can use the following Python script as a starting point. This script uses the `google-auth` and `google-auth-oauthlib` libraries to send an email using Gmail, and the `requests` library to send a webhook and receive a response. Make sure to replace `YOUR_GMAIL_APP_PASSWORD` and `YOUR_WEBHOOK_URL` with your actual credentials and webhook URL.

```python
import os
import base64
import json
import requests
from google.oauth2 import service_account
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# Set up Gmail API credentials
creds = service_account.Credentials.from_service_account_file(
    'path/to/your/service_account.json')
service = build('gmail', 'v1', credentials=creds)

# Define email message
message = {
    'raw': base64.urlsafe_b64encode(
        b'From: your_email@example.com\r\n'
        b'To: sanyam0605@gmail.com\r\n'
        b'Subject: Request for SQL Query Output\r\n\r\n'
   